In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


25/03/02 08:35:43 WARN Utils: Your hostname, Chriss-MacBook-Pro-7.local resolves to a loopback address: 127.0.0.1; using 10.42.0.101 instead (on interface en0)
25/03/02 08:35:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 08:35:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!mkdir -p data/raw/yellow/2024/10
!cd data/raw/yellow/2024/10
!wget wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
!cd ../../../../..

In [4]:
yellow_2024_10 = spark.read \
    .parquet("data/raw/yellow/2024/10")

In [ ]:
yellow_2024_10.schema

In [ ]:
yellow_2024_10.printSchema()

In [5]:
y10p = yellow_2024_10.repartition(4)

In [ ]:
y10p.write.parquet("homework/y2024/q2")

In [6]:
from pyspark.sql.functions import dayofmonth
filtered = y10p.filter(dayofmonth(y10p.tpep_pickup_datetime) == '15')

In [7]:
filtered.take(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 15, 0, 43, 57), tpep_dropoff_datetime=datetime.datetime(2024, 10, 15, 0, 45, 14), passenger_count=1, trip_distance=0.29, RatecodeID=5, store_and_fwd_flag='N', PULocationID=138, DOLocationID=223, payment_type=1, fare_amount=65.0, extra=5.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=72.75, congestion_surcharge=0.0, Airport_fee=1.75),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 15, 8, 29, 29), tpep_dropoff_datetime=datetime.datetime(2024, 10, 15, 8, 45, 25), passenger_count=0, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=79, DOLocationID=170, payment_type=1, fare_amount=14.2, extra=2.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=18.2, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 15, 15, 17, 23), tpep_dropoff_datetime=datetime.

In [14]:
filtered.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-15 00:43:57|  2024-10-15 00:45:14|              1|         0.29|         5|                 N|         138|         223|           1|       65.0|  5.0|    0.0|       0.

In [8]:
filtered.count()

128893

In [19]:
from pyspark.sql.functions import date_diff


In [9]:
from pyspark.sql.functions import max_by, to_timestamp

In [10]:
yellow_2024_10\
    .withColumn("trip_duration", \
    (to_timestamp(y10p.tpep_dropoff_datetime).cast('long') \
    - to_timestamp(y10p.tpep_pickup_datetime).cast('long'))\
    / 60 / 60)\
    .agg(max_by('trip_duration', 'trip_duration'))\
    .show()

+------------------------------------+
|max_by(trip_duration, trip_duration)|
+------------------------------------+
|                   162.6177777777778|
+------------------------------------+



25/03/02 10:44:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 382755 ms exceeds timeout 120000 ms
25/03/02 10:44:43 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/02 10:44:50 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [5]:
zones = spark.read\
    .option('header', True)\
    .option('inferSchema', True)\
    .csv('taxi_zone_lookup.csv')
zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [6]:
zones.createOrReplaceTempView('zones')

In [7]:
spark.sql("""
          
select * from zones
    
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [8]:
yellow_2024_10.createOrReplaceTempView("yellow_2024_10")

In [22]:
spark.sql("""
    select count(1), Zone from yellow_2024_10
    left outer join zones on (PULocationID = LocationID)
    group by Zone
    order by count(1)
""").show()

+--------+--------------------+
|count(1)|                Zone|
+--------+--------------------+
|       1|Governor's Island...|
|       2|       Rikers Island|
|       2|       Arden Heights|
|       3|         Jamaica Bay|
|       3| Green-Wood Cemetery|
|       4|Charleston/Totten...|
|       4|   Rossville/Woodrow|
|       4|       West Brighton|
|       4|Eltingville/Annad...|
|       4|       Port Richmond|
|       6|         Great Kills|
|       6|        Crotona Park|
|       7|Heartland Village...|
|       7|     Mariners Harbor|
|       9|Saint George/New ...|
|       9|             Oakwood|
|      10|       Broad Channel|
|      10|New Dorp/Midland ...|
|      12|         Westerleigh|
|      12|     Pelham Bay Park|
+--------+--------------------+
only showing top 20 rows



25/03/02 02:30:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 294269 ms exceeds timeout 120000 ms
25/03/02 02:30:26 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/02 02:30:32 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$